### 제품 정보 가져오는 크롤링 코드
이 노트북의 코드는 M5_Idol_lyrics 레파지토리의 url_crawling, melon_crawling 노트북들을 참고하였습니다.
@kavin, @pje1740 님 감사합니다.

In [13]:
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
import requests
import json
import re
import pandas as pd
import numpy as np

In [14]:
from urllib import request
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys

In [15]:
# 수집을 위한 빈 리스트 생성

category_main = []  # 카테고리_대분류
category_small = []  # 카테고리_소분류
product_name = []  # 제품명
price = []  # 가격_판매가(고정가. 최저가 없이 하나만 있음)
price_curr = []  # 가격_판매가(현재가)
price_lowest = []  # 가격_최저가 
selling_code = []  # 판매코드
color = []  # 색상
age = []  # 사용연령
release_date = []  #동일모델의 출시년월
manufacturer = []  # 제조자
satisfaction = []  # 고객만족도
review_count = []  # 상품평 갯수
review = []  # 상품평

## 크롤러

In [16]:
product_list = pd.read_csv('../data/CategoryID_ProductName.csv')

In [17]:
product_list = product_list.rename(index=str, columns={"카테고리ID": "category_ID", "제품명": "product_name"})
product_list.shape

(5934, 2)

In [18]:
product_list.head()

,category_ID,product_name
0,C10100030001,베이블레이드초Z 리바이브피닉스
1,C10100030001,베이블레이드갓 커스터마이즈
2,C10100030001,베이블레이드초Z 올인원세트
3,C10100030001,베이블레이드 갓배틀세트
4,C10100030001,베이블레이드초Z 랜덤부스터11


In [19]:
# 중복값 제거(검색은 유니크한 제품만 하면 되므로)
product_list = product_list.drop_duplicates(subset='product_name', keep='first', inplace=False)

In [21]:
product_list.shape

(5020, 2)

In [22]:
names = product_list.product_name
names[0]

'베이블레이드초Z 리바이브피닉스'

In [4]:
def list_clear():
    category_main.clear()
    category_small.clear()
    product_name.clear()
    price.clear()
    price_curr.clear()
    price_lowest.clear()
    selling_code.clear()
    color.clear()
    age.clear()
    release_date.clear()
    manufacturer.clear()
    satisfaction.clear()
    review_count.clear()
    review.clear()

In [5]:
def append_list():   
    product_name.append(driver.find_element_by_xpath('//*[@id="document"]/article/div[1]/div[1]/h3').text)
    selling_code.append(driver.find_element_by_xpath('//*[@id="ItemSrcmkCD"]').text)
    color.append(driver.find_element_by_xpath('//*[@id="itemDetail"]/div[1]/table/tbody/tr[5]/td').text)
    age.append(driver.find_element_by_xpath('//*[@id="itemDetail"]/div[1]/table/tbody/tr[7]/td').text)
    release_date.append(driver.find_element_by_xpath('//*[@id="itemDetail"]/div[1]/table/tbody/tr[8]/td').text)
    manufacturer.append(driver.find_element_by_xpath('//*[@id="itemDetail"]/div[1]/table/tbody/tr[9]/td').text)
    
    
    # 카테고리가 '기획전' 등 기타 카테고리인 경우가 있음    
    try:
        category_main.append(driver.find_element_by_xpath('//*[@id="list_prod_top"]/a[3]').text)
    except NoSuchElementException as e:
        category_main.append('기타')
        
    try:
        category_small.append(driver.find_element_by_xpath('//*[@id="list_prod_top"]/a[4]').text)
    except NoSuchElementException as e:
        category_main.append('기타')
        
    
    # 판매가가 고정가/ 현재가,최저가 로 나뉘어 있어서 있는 것만 담아준다.
    
    try:
        price.append(driver.find_element_by_xpath('//*[@id="ItemSalePrc"]').text)
    except NoSuchElementException as e:
        price.append('')
                                                     
    try:
        price_curr.append(driver.find_element_by_xpath('//*[@id="ItemCurrSalePrc"]').text)
    except NoSuchElementException as e:
        price_curr.append('')
        
    try:
        price_lowest.append(driver.find_element_by_xpath('//*[@id="ItemCurrSalePrc"]').text)
    except NoSuchElementException as e:
        price_lowest.append('') 
        
        
    # 고객 만족도와 상품평 없는 경우
    
    try:
        satisfaction.append(driver.find_element_by_xpath('//*[@id="document"]/article/div[2]/div[3]/div[2]/div[1]/span[3]').text)
    except NoSuchElementException as e:
        satisfaction.append('') 
        
    try:
        review_count.append(driver.find_element_by_xpath('//*[@id="document"]/article/div[2]/div[3]/div[2]/div[2]/a/span/strong').text)
    except NoSuchElementException as e:
        review_count.append('')
        
# 'http://toysrus.lottemart.com/product/ProductDetail.do?CategoryID=C00300020100&ProductCD=8801198125306&cPage=1&selBoard=REG_DATE&selBoard2=ALL_RECOMM#prdComment'

In [15]:
path=r'../../shoppingmall/chromedriver'
driver=webdriver.Chrome(path)
driver.get("http://toysrus.lottemart.com/index.do;jsessionid=JRTV4GA2AXOTOWPvFI70goBv22rvNVdyg5kKtiH9DalebEQ108mVkLXQt1qnql9Y.lmtrwas01_servlet_engine1")

In [16]:
list_clear()
i = 0
for names[i] in names:
    search = driver.find_element_by_xpath('//*[@id="searchTerm"]')
    search.send_keys(Keys.CONTROL + "a")
    search.clear()
    search.send_keys(names[i])
    
    # 검색버튼 클릭
    driver.find_element_by_xpath('//*[@id="mainSearchForm"]/button').click()
    
    # 제품 클릭
    driver.find_element_by_xpath('//*[@id="document"]/div[4]/ul/li/div[1]/a').click()
    
    append_list()
#     print(color)
#     append_review()
#     search.send_keys(Keys.CONTROL + "a")
#     search.clear()
#     i =+ 1

['빨강,골드외']
['빨강,골드외', '레드,블랙,옐로우외']
['빨강,골드외', '레드,블랙,옐로우외', '빨강,보라외']


NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=70.0.3538.110)
  (Driver info: chromedriver=2.41.578706 (5f725d1b4f0a4acbf5259df887244095596231db),platform=Mac OS X 10.14.2 x86_64)


### 테스트

In [1]:
# 테스트
# search = driver.find_element_by_xpath('//*[@id="searchTerm"]')
# search.send_keys(Keys.CONTROL + "a")
# search.clear()
# search.send_keys('[캐스돈] CASDON 네비운전놀이 (CD06347)')
    
#     # 검색버튼 클릭
# driver.find_element_by_xpath('//*[@id="mainSearchForm"]/button').click()

#     # 제품 클릭
# driver.find_element_by_xpath('//*[@id="document"]/div[4]/ul/li/div[1]/a').click()
    
# append_list()

In [17]:
column_names = ['category_main', 'category_small', 'product_name', 'price', 'price_curr', 'price_lowest', 'selling_code', 'color',
             'age', 'release_date', 'manufacturer', 'satisfaction', 'review_count', 'review']
column_v = [category_main, category_small, product_name, price, price_curr, price_lowest, selling_code, color,
             age, release_date, manufacturer, satisfaction, review_count, review]

df = pd.DataFrame(column_v).T
df.columns = column_names

In [18]:
df.head()

,category_main,category_small,product_name,price,price_curr,price_lowest,selling_code,color,age,release_date,manufacturer,satisfaction,review_count,review
0,기타,None,베이블레이드초Z 리바이브피닉스,,"21,300","21,300",8801198121179,"빨강,골드외",6세이상,2018년9월,(주)영실업,9.8,98,None
1,기타,None,㉥ 베이블레이드갓 커스터마이즈,,"49,200","49,200",8801198120981,"레드,블랙,옐로우외",6세이상,2018년01월,영실업,9.8,153,None
2,기타,None,㉥ 베이블레이드초Z 올인원세트,"51,700",,,8801198125375,"빨강,보라외",6세이상,2018년9월,(주)영실업,9.8,8,None
3,기타,None,㉥ 베이블레이드 갓배틀세트,None,None,None,8801198125306,"블루,레드외",6세이상,2017년10월,영실업,None,None,None
4,기타,None,None,None,None,None,None,None,None,None,None,None,None,None
5,기타,None,None,None,None,None,None,None,None,None,None,None,None,None


In [ ]:
df.to_csv('product_info.csv', index=False)

In [143]:
# df[df.price_curr == '']

,category_main,category_small,product_name,price,price_curr,price_lowest,selling_code,color,age,release_date,manufacturer,satisfaction,review_count,review
0,유아 Toys,자동차놀이,[품절] [캐스돈] CASDON 네비운전놀이 (CD06347),"44,000",,,L000000553188,검정 외,만 3세 이상,201311,CASDON/큐이디㈜,,,None
1,유아 Toys,자동차놀이,[품절] [캐스돈] CASDON 네비운전놀이 (CD06347),"44,000",,,L000000553188,검정 외,만 3세 이상,201311,CASDON/큐이디㈜,,,None


### 리뷰 가져오려고 짜던 코드

In [ ]:
# get_review(p_count,1,-1)

In [27]:
# def get_review(whole,last,decrease):
#     p_count = int(driver.find_element_by_xpath('//*[@id="prdComment"]/div[3]/div[1]/button[1]/span').text)    
#     for i in range(whole,last,decrease):
#         a1 = '//*[@id="'
#         a2 = '"]/td[3]/a'
#         try:
#             driver.find_element_by_xpath(a1 + str(i) + a2).click()
#         except NoSuchElementException as e:
#             review.append('')
            
#         p1 = '//*[@id="r_'
#         p2 = '"]/td/div/p[2]'
    
#         p1_no_picture = '//*[@id="r_'
#         p2_no_picture = '"]/td/div/p'
    
#         try:
#             review.append(driver.find_element_by_xpath(p1 + str(i) + p2).text)
#         except NoSuchElementException as e:
#             review.append(driver.find_element_by_xpath(p1_no_picture + str(i) + p2_no_picture).text)
#         except NoSuchElementException as e:
#             review.append('')    
            
#         try:
#             review.append(driver.find_element_by_xpath(p1_no_picture + str(i) + p2_no_picture).text)
#         except NoSuchElementException as e:
#             review.append(driver.find_element_by_xpath(p1 + str(i) + p2).text)
#         except NoSuchElementException as e:
#             review.append('')

In [13]:
# url = 'http://toysrus.lottemart.com/product/ProductDetail.do?CategoryID=C00300020100&ProductCD=8801198125306&cPage=1&selBoard=REG_DATE&selBoard2=ALL_RECOMM#prdComment'
# # url = driver.current_url
# id = url.replace('http://toysrus.lottemart.com/product/ProductDetail.do?CategoryID=C00300020100&ProductCD=',
#                   '')
# id = id.replace('&cPage=1&selBoard=REG_DATE&selBoard2=ALL_RECOMM#prdComment','')
# id

'8801198125306'

In [ ]:
# len(product_id)

In [148]:
# def append_review(): 
    
#     a1 = '//*[@id="'
#     a2 = '"]/td[3]/a'
#     try:
#         driver.find_element_by_xpath(a1 + str(i) + a2).click()
#     except NoSuchElementException as e:
#         review.append('')
    
#     p1 = '//*[@id="r_'
#     p2 = '"]/td/div/p[2]'
    
#     p1_no_picture = '//*[@id="r_'
#     p2_no_picture = '"]/td/div/p'
    
#     try:
#         review.append(driver.find_element_by_xpath(p1 + str(i) + p2).text)
#     except NoSuchElementException as e:
#         review.append(driver.find_element_by_xpath(p1_no_picture + str(i) + p2_no_picture).text)
        
#     print(review)
#     print('done')

In [37]:
# def append_list():   
#     product_name.append(driver.find_element_by_xpath('//*[@id="document"]/article/div[1]/div[1]/h3').text)
#     selling_code.append(driver.find_element_by_xpath('//*[@id="ItemSrcmkCD"]').text)
#     color.append(driver.find_element_by_xpath('//*[@id="itemDetail"]/div[1]/table/tbody/tr[5]/td').text)
#     age.append(driver.find_element_by_xpath('//*[@id="itemDetail"]/div[1]/table/tbody/tr[7]/td').text)
#     release_date.append(driver.find_element_by_xpath('//*[@id="itemDetail"]/div[1]/table/tbody/tr[8]/td').text)
#     manufacturer.append(driver.find_element_by_xpath('//*[@id="itemDetail"]/div[1]/table/tbody/tr[9]/td').text)
    
    
#     # 카테고리가 '기획전' 등 기타 카테고리인 경우가 있음    
#     try:
#         category_main.append(driver.find_element_by_xpath('//*[@id="list_prod_top"]/a[3]').text)
#     except NoSuchElementException as e:
#         category_main.append('기타')
        
#     try:
#         category_small.append(driver.find_element_by_xpath('//*[@id="list_prod_top"]/a[4]').text)
#     except NoSuchElementException as e:
#         category_main.append('기타')
        
    
#     # 판매가가 고정가/ 현재가,최저가 로 나뉘어 있어서 있는 것만 담아준다.
    
#     try:
#         price.append(driver.find_element_by_xpath('//*[@id="ItemSalePrc"]').text)
#     except NoSuchElementException as e:
#         price.append('')
                                                     
#     try:
#         price_curr.append(driver.find_element_by_xpath('//*[@id="ItemCurrSalePrc"]').text)
#     except NoSuchElementException as e:
#         price_curr.append('')
        
#     try:
#         price_lowest.append(driver.find_element_by_xpath('//*[@id="ItemCurrSalePrc"]').text)
#     except NoSuchElementException as e:
#         price_lowest.append('') 
        
        
#     # 고객 만족도와 상품평 없는 경우
    
#     try:
#         satisfaction.append(driver.find_element_by_xpath('//*[@id="document"]/article/div[2]/div[3]/div[2]/div[1]/span[3]').text)
#     except NoSuchElementException as e:
#         satisfaction.append('') 
        
#     try:
#         review_count.append(driver.find_element_by_xpath('//*[@id="document"]/article/div[2]/div[3]/div[2]/div[2]/a/span/strong').text)
#     except NoSuchElementException as e:
#         review_count.append('')
        
#     # 상품평 가져오기
# #     index = 1
#     p_count = int(driver.find_element_by_xpath('//*[@id="prdComment"]/div[3]/div[1]/button[1]/span').text)
#     for x in range(p_count//10+1):
#         for i in range(p_count,1,-1):
#             a1 = '//*[@id="'
#             a2 = '"]/td[3]/a'
#             try:
#                 driver.find_element_by_xpath(a1 + str(i) + a2).click()
#             except NoSuchElementException as e:
#                 review.append('')

#             p1 = '//*[@id="r_'
#             p2 = '"]/td/div/p[2]'

#             p1_no_picture = '//*[@id="r_'
#             p2_no_picture = '"]/td/div/p'

#             try:
#                 review.append(driver.find_element_by_xpath(p1 + str(i) + p2).text)
#             except NoSuchElementException as e:
#                 review.append(driver.find_element_by_xpath(p1_no_picture + str(i) + p2_no_picture).text)
# #             except NoSuchElementException as e:
# #                 review.append('')    

#             try:
#                 review.append(driver.find_element_by_xpath(p1_no_picture + str(i) + p2_no_picture).text)
#             except NoSuchElementException as e:
#                 review.append(driver.find_element_by_xpath(p1 + str(i) + p2).text)
# #             except NoSuchElementException as e:
# #                 review.append('')

#             print(review)


#             if (i % 10 == (p_count % 10) +1):
#     #             index += 1
#     #             break

#                 for n in range(1, 10):

#                     if (driver.find_element_by_xpath('//*[@id="pagingDiv"]/strong').text) == '1':
#                         driver.find_element_by_xpath('//*[@id="pagingDiv"]/a[1]').click()
                        
#                     else:
#                         driver.find_element_by_xpath('//*[@id="pagingDiv"]/a[3]').click()
                        
#                     p_count_page2 = p_count-10
#                     get_review(p_count_page2,p_count_page2-9,-1)
                    
#                     for n in range(3, 10):
#                         if n <= 10:
#                             next1 = '//*[@id="pagingDiv"]/a['
#                             next2 = ']'
#                             driver.find_element_by_xpath(next1 + str(n) + next2).click()
#                             get_review(p_count_page2-20,p_count_page2-29,-1)

#                         elif n > 10:
#                             for n2 in range(3, p_count % 10):
#                                 driver.find_element_by_xpath(next1 + str(n2) + next2).click()

# #                     get_review()

# #             first = 'http://toysrus.lottemart.com/product/ProductDetail.do?CategoryID=C00300020100&ProductCD=8801198125306&cPage='
# #             second = '&selBoard=REG_DATE&selBoard2=ALL_RECOMM#prdComment'      
# #             # 이거 하려면 프로덕트CD 를 전부 알아서 그 안에서 for 문을 돌려야 함..
            
# #             for n in range(2, p_count // 10 + 1):
# #                 while True:
# #                     driver.get(first+str(n)+second)      
#     print(review)
# #     print('done')
    
# #     print(category_main)
# #     print('done')
        
# # 'http://toysrus.lottemart.com/product/ProductDetail.do?CategoryID=C00300020100&ProductCD=8801198125306&cPage=1&selBoard=REG_DATE&selBoard2=ALL_RECOMM#prdComment'

In [44]:
# def append_list():   
#     product_name.append(driver.find_element_by_xpath('//*[@id="document"]/article/div[1]/div[1]/h3').text)
#     selling_code.append(driver.find_element_by_xpath('//*[@id="ItemSrcmkCD"]').text)
#     color.append(driver.find_element_by_xpath('//*[@id="itemDetail"]/div[1]/table/tbody/tr[5]/td').text)
#     age.append(driver.find_element_by_xpath('//*[@id="itemDetail"]/div[1]/table/tbody/tr[7]/td').text)
#     release_date.append(driver.find_element_by_xpath('//*[@id="itemDetail"]/div[1]/table/tbody/tr[8]/td').text)
#     manufacturer.append(driver.find_element_by_xpath('//*[@id="itemDetail"]/div[1]/table/tbody/tr[9]/td').text)
    
    
#     # 카테고리가 '기획전' 등 기타 카테고리인 경우가 있음    
#     try:
#         category_main.append(driver.find_element_by_xpath('//*[@id="list_prod_top"]/a[3]').text)
#     except NoSuchElementException as e:
#         category_main.append('기타')
        
#     try:
#         category_small.append(driver.find_element_by_xpath('//*[@id="list_prod_top"]/a[4]').text)
#     except NoSuchElementException as e:
#         category_main.append('기타')
        
    
#     # 판매가가 고정가/ 현재가,최저가 로 나뉘어 있어서 있는 것만 담아준다.
    
#     try:
#         price.append(driver.find_element_by_xpath('//*[@id="ItemSalePrc"]').text)
#     except NoSuchElementException as e:
#         price.append('')
                                                     
#     try:
#         price_curr.append(driver.find_element_by_xpath('//*[@id="ItemCurrSalePrc"]').text)
#     except NoSuchElementException as e:
#         price_curr.append('')
        
#     try:
#         price_lowest.append(driver.find_element_by_xpath('//*[@id="ItemCurrSalePrc"]').text)
#     except NoSuchElementException as e:
#         price_lowest.append('') 
        
        
#     # 고객 만족도와 상품평 없는 경우
    
#     try:
#         satisfaction.append(driver.find_element_by_xpath('//*[@id="document"]/article/div[2]/div[3]/div[2]/div[1]/span[3]').text)
#     except NoSuchElementException as e:
#         satisfaction.append('') 
        
#     try:
#         review_count.append(driver.find_element_by_xpath('//*[@id="document"]/article/div[2]/div[3]/div[2]/div[2]/a/span/strong').text)
#     except NoSuchElementException as e:
#         review_count.append('')
        
#     # 상품평 가져오기
# #     index = 1
#     p_count = int(driver.find_element_by_xpath('//*[@id="prdComment"]/div[3]/div[1]/button[1]/span').text)
#     for x in range():
#     for i in range(p_count,1,-1):
#         a1 = '//*[@id="'
#         a2 = '"]/td[3]/a'
#         try:
#             driver.find_element_by_xpath(a1 + str(i) + a2).click()
#         except NoSuchElementException as e:
#             review.append('')
            
#         p1 = '//*[@id="r_'
#         p2 = '"]/td/div/p[2]'
    
#         p1_no_picture = '//*[@id="r_'
#         p2_no_picture = '"]/td/div/p'
    
#         try:
#             review.append(driver.find_element_by_xpath(p1 + str(i) + p2).text)
#         except NoSuchElementException as e:
#             review.append(driver.find_element_by_xpath(p1_no_picture + str(i) + p2_no_picture).text)
#         except NoSuchElementException as e:
#             review.append('')    
            
#         try:
#             review.append(driver.find_element_by_xpath(p1_no_picture + str(i) + p2_no_picture).text)
#         except NoSuchElementException as e:
#             review.append(driver.find_element_by_xpath(p1 + str(i) + p2).text)
#         except NoSuchElementException as e:
#             review.append('')
            
#         print(review)
    
        
#         if (i % 10 == (p_count % 10) +1):
# #             index += 1
# #             break
            
#             for n in range(1, 10):

#                 if (driver.find_element_by_xpath('//*[@id="pagingDiv"]/strong').text) == '1':
#                     driver.find_element_by_xpath('//*[@id="pagingDiv"]/a[1]').click()
#                 else:
#                     driver.find_element_by_xpath('//*[@id="pagingDiv"]/a[3]').click()
                
#                 get_review()
                
#                 for n in range(3, 10):
#                     if n <= 10:
#                         next1 = '//*[@id="pagingDiv"]/a['
#                         next2 = ']'
#                         driver.find_element_by_xpath(next1 + str(n) + next2).click()
            
#                     elif n > 10:
#                         for n2 in range(3, p_count % 10):
#                             driver.find_element_by_xpath(next1 + str(n2) + next2).click()
                            
#                 get_review()

# #             first = 'http://toysrus.lottemart.com/product/ProductDetail.do?CategoryID=C00300020100&ProductCD=8801198125306&cPage='
# #             second = '&selBoard=REG_DATE&selBoard2=ALL_RECOMM#prdComment'      
# #             # 이거 하려면 프로덕트CD 를 전부 알아서 그 안에서 for 문을 돌려야 함..
            
# #             for n in range(2, p_count // 10 + 1):
# #                 while True:
# #                     driver.get(first+str(n)+second)      
#     print(review)
# #     print('done')
    
# #     print(category_main)
# #     print('done')
        
# # 'http://toysrus.lottemart.com/product/ProductDetail.do?CategoryID=C00300020100&ProductCD=8801198125306&cPage=1&selBoard=REG_DATE&selBoard2=ALL_RECOMM#prdComment'